# Preprocessing: ICA

The workhorse functions `clean_withar_local` and `get_ica_weights` are defined in `..\preprocess.py`. This allows to call them also from bash scripts to run them in the background to precalculate their outputs (which - depending on your setup - will take a while). 
You can then here set all `*_from_disc` arguments to `True` and load the precalculated values directly. 
This script has some interactive steps:
- marking stretches of clean EOG data (can be skipped if loaded from disk) 
- inspecting ICA components
- inspecting results of the ICA cleaning

So it won't run over all participants without your interaction. Good to look at some data anyway.
I recommend precalculating the ICA weights incl. the first round of Autoreject in separate script (use `..\utils\03_preprocess-ica-ar.py` for this) and then just load the values here. 

In [1]:
from os import path as op
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mne
from mne.preprocessing import create_eog_epochs
import autoreject

from library import config, helpers, preprocess

In [2]:
%matplotlib qt

%load_ext autoreload
%autoreload 2

In [3]:
def get_eog_epochs_clean(data_raw,
                         eogannot_from_disc = False,
                         save_eogannot_to_disc = True,
                         eogannot_path=config.paths['03_preproc-ica-eog']):
    mark_data = data_raw.copy()

    if eogannot_from_disc:
        fname = op.join(eogannot_path, subID + '-eog-annot.fif')
        annot = mne.read_annotations(fname)
        mark_data.set_annotations(annot)
    else:
        # Mark stretches (at least 1) of data with clean EOG
        mark_data.pick_types(eog=True, eeg=False).filter(l_freq=1, h_freq=None,
                                                         picks=['eog'],
                                                         verbose=False).\
                                                  plot(duration=180,
                                                       scalings={'eog': 850e-6},
                                                       #remove_dc=True,
                                                       block=True)
        if save_eogannot_to_disc:
            fname = op.join(eogannot_path, subID + '-eog-annot.fif')
            mark_data.annotations.save(fname, overwrite=True)

    # Calculate clean epochs locked to EOG-peaks (!name the annotation `clean_eeg`!):
    raws = [idx for idx in mark_data.annotations if idx['description'] == 'clean_eog']
    t0 = raws[0]['onset']
    t1 = t0 + raws[0]['duration']
    raw_eog = data_raw.copy().crop(t0, t1)

    if len(raws)>1:
        for r_ in raws[1:]:
            t0 = r_['onset']
            t1 = t0 + r_['duration']
            raw_eog.append(data_raw.copy().crop(t0, t1))
            
    # Heuristically determine a robust threshold for EOG peak detection
    tmp = raw_eog.copy().load_data().\
                         pick_channels(['VEOG', 'HEOG']).\
                         filter(l_freq=1, h_freq=10, picks=['eog'], verbose=False)
    
    tmp_epo = mne.make_fixed_length_epochs(tmp, preload=True)

    mean_threshs = np.mean(tmp_epo.get_data().squeeze().ptp(axis=0) / 4, axis=1)
    thresh_dict = {ch:thresh for ch, thresh in zip(tmp_epo.ch_names, mean_threshs)}

    epochs_veog = create_eog_epochs(raw_eog,
                                    ch_name='VEOG',
                                    baseline=(None,None),
                                    thresh=thresh_dict['VEOG'],
                                    verbose=False)

    print(f'Created {len(epochs_veog)} VEOG epochs.')
    if (len(epochs_veog) < 50):
        print('Not really a lot. This might be a problem! Consider marking longer stretches of clean EOG data.')
        print('########################')

    epochs_heog = create_eog_epochs(raw_eog,
                                    ch_name='HEOG',
                                    baseline=(None,None),
                                    thresh=thresh_dict['HEOG'],
                                    verbose=False)

    print(f'Created {len(epochs_heog)} HEOG epochs.')
    if (len(epochs_heog) < 50):
        print('Not really a lot. This might be a problem! Consider marking longer stretches of clean EOG data.')
        print('########################')

    return epochs_veog, epochs_heog


def vis_compare_ica(data_before, data_after, show_data_before=False, block=True):
    # visual comparison:
    picks=['eeg','eog']
    if show_data_before:
        old = data_before.copy().\
        apply_baseline((None,None)).\
        plot(scalings=dict(eeg=50e-6),
                               n_epochs=15,
                               n_channels=30,
                               picks=picks) 
    new = data_after.copy().\
    apply_baseline((None,None)).\
    plot(scalings=dict(eeg=50e-6),
                          n_epochs=15,
                          n_channels=30,
                          picks=picks,
                          block=block)

In [4]:
# Loop over all subjects:
sub_list = np.setdiff1d(np.arange(1,config.n_subjects_total+1), config.ids_missing_subjects)
sub_list_str = ['VME_S%02d' % sub for sub in sub_list]

## to run a single subject, modify and uncomment one of the following lines:
# sub_list_str = ['VME_S27']

for subID in sub_list_str:
    data_raw = helpers.load_data(subID + '-prepared',
                                        config.paths['01_prepared'],
                                        append='-raw').load_data()
    data_forICA = helpers.load_data(subID + '-ica',
                                        op.join(config.paths['02_epochs'], '0.01', 'ica'),
                                        append='-epo')

    # clean it with autoreject local to remove bad epochs for better ICA fit:
    data_forAR = data_forICA.copy().apply_baseline((-config.times_dict['bl_dur_erp'],0)) # AR does not perform well on non-baseline corrected data

    _, ar, reject_log = preprocess.clean_with_ar_local(subID,
                                                       data_forAR,
                                                       ar_from_disc=True,
                                                       save_to_disc=False,
                                                       ar_path=config.paths['03_preproc-ica-ar'])

    # Get ICA weights
    ica = preprocess.get_ica_weights(subID,
                                      data_forICA[~reject_log.bad_epochs],
                                      picks=None,
                                      reject=None,
                                      method='picard',
                                      fit_params=None,
                                      ica_from_disc=True,
                                      save_to_disc=False,
                                      ica_path=config.paths['03_preproc-ica'])

    # Calculate EOG epochs from clean stretches of data:
    epochs_veog, epochs_heog = get_eog_epochs_clean(data_raw,
                                                    eogannot_from_disc=True,
                                                    save_eogannot_to_disc=False,
                                                    eogannot_path=config.paths['03_preproc-ica-eog'])

    # Reject ICA components with high correlations with V/HEOG channel
    threshold = 0.9
    indices_veog, scores_veog = ica.find_bads_eog(epochs_veog,
                                                  ch_name='VEOG',
                                                  measure='correlation',
                                                  threshold=threshold)

    indices_heog, scores_heog = ica.find_bads_eog(epochs_heog,
                                                  ch_name='HEOG',
                                                  measure='correlation',
                                                  threshold=threshold)
    
#     ica.plot_scores([scores_veog, scores_heog],
#                     labels=['VEOG', 'HEOG'],
#                     exclude=indices_veog + indices_heog)

    exclude = list(np.unique(indices_veog + indices_heog))
    ica.exclude = exclude

#    fig = ica.plot_components(range(25), inst=data_forICA)
#    plt.show(block = True)

    # kick out components from data to clean and save those to disc:
    for epo_part in ['cue', 'fulllength', 'stimon']:

        data_pre = helpers.load_data(f'{subID}-{epo_part}',
                                     op.join(config.paths['02_epochs'], '0.1', epo_part),
                                     append='-epo',
                                     verbose=False)

        data_post = ica.apply(data_pre.copy())

        helpers.save_data(data_post,
                          f'{subID}-{epo_part}-postICA',
                          op.join(config.paths['03_preproc-ica'], 'cleaneddata', '0.1', epo_part),
                          append='-epo')
        
#     evo = data_post.copy().average().apply_baseline((-config.times_dict['bl_dur_erp'], 0))
#     evo.plot()
#     data_post.plot_psd()
#     vis_compare_ica(data_pre, data_post, show_data_before=True, block=True)

Opening raw data file /raven/ptmp/fklotzsche/Experiments/vMemEcc/Data2022/DataMNE/EEG/01_prepared/VME_S01-prepared-raw.fif...
    Range : 0 ... 4353149 =      0.000 ...  8706.298 secs
Ready.
Reading 0 ... 4353149  =      0.000 ...  8706.298 secs...
Reading /raven/ptmp/fklotzsche/Experiments/vMemEcc/Data2022/DataMNE/EEG/02_epochs/0.01/ica/VME_S01-ica-epo.fif ...
    Found the data of interest:
        t =    -600.00 ...    2300.00 ms
        0 CTF compensation matrices available
Not setting metadata
Not setting metadata
720 matching events found
No baseline correction applied
0 projection items activated
Applying baseline correction (mode: mean)


  0%|          | Repairing epochs : 0/720 [00:00<?,       ?it/s]

Dropped 32 epochs: 13, 45, 114, 124, 139, 154, 247, 268, 364, 390, 401, 412, 413, 414, 418, 421, 484, 486, 491, 511, 515, 528, 540, 559, 565, 587, 615, 630, 689, 690, 700, 704
Reading /raven/ptmp/fklotzsche/Experiments/vMemEcc/Data2022/DataMNE/EEG/03_preproc/ica/VME_S01-ica.fif ...
Now restoring ICA solution ...
Ready.
Not setting metadata
Not setting metadata
546 matching events found
No baseline correction applied
0 projection items activated
Loading data for 546 events and 500 original time points ...
4 bad epochs dropped
Created 104 VEOG epochs.
Created 71 HEOG epochs.
Using EOG channel: VEOG
Using EOG channel: HEOG
Applying ICA to Epochs instance
    Transforming to ICA space (62 components)
    Zeroing out 2 ICA components
    Projecting back using 62 PCA components
creating dir: /raven/ptmp/fklotzsche/Experiments/vMemEcc/Data2022/DataMNE/EEG/03_preproc/ica/cleaneddata/0.1/cue
Applying ICA to Epochs instance
    Transforming to ICA space (62 components)
    Zeroing out 2 ICA comp

  0%|          | Repairing epochs : 0/720 [00:00<?,       ?it/s]

Dropped 5 epochs: 114, 236, 250, 297, 529
Reading /raven/ptmp/fklotzsche/Experiments/vMemEcc/Data2022/DataMNE/EEG/03_preproc/ica/VME_S02-ica.fif ...
Now restoring ICA solution ...
Ready.
Not setting metadata
Not setting metadata
568 matching events found
No baseline correction applied
0 projection items activated
Loading data for 568 events and 500 original time points ...
3 bad epochs dropped
Created 114 VEOG epochs.
Created 99 HEOG epochs.
Using EOG channel: VEOG
Using EOG channel: HEOG
Applying ICA to Epochs instance
    Transforming to ICA space (62 components)
    Zeroing out 1 ICA component
    Projecting back using 62 PCA components
Applying ICA to Epochs instance
    Transforming to ICA space (62 components)
    Zeroing out 1 ICA component
    Projecting back using 62 PCA components
Applying ICA to Epochs instance
    Transforming to ICA space (62 components)
    Zeroing out 1 ICA component
    Projecting back using 62 PCA components
Opening raw data file /raven/ptmp/fklotzsche

...

Dropped 26 epochs: 17, 35, 41, 45, 67, 244, 293, 310, 319, 321, 327, 343, 345, 348, 354, 362, 413, 424, 455, 476, 545, 560, 562, 607, 679, 690
Reading /raven/ptmp/fklotzsche/Experiments/vMemEcc/Data2022/DataMNE/EEG/03_preproc/ica/VME_S25-ica.fif ...
Now restoring ICA solution ...
Ready.
Not setting metadata
Not setting metadata
425 matching events found
No baseline correction applied
0 projection items activated
Loading data for 425 events and 500 original time points ...
7 bad epochs dropped
Created 113 VEOG epochs.
Created 11 HEOG epochs.
Not really a lot. This might be a problem! Consider marking longer stretches of clean EOG data.
########################
Using EOG channel: VEOG
Using EOG channel: HEOG
Applying ICA to Epochs instance
    Transforming to ICA space (62 components)
    Zeroing out 2 ICA components
    Projecting back using 62 PCA components
Applying ICA to Epochs instance
    Transforming to ICA space (62 components)
    Zeroing out 2 ICA components
    Projecting bac

  0%|          | Repairing epochs : 0/720 [00:00<?,       ?it/s]

Dropped 8 epochs: 119, 138, 140, 212, 255, 485, 702, 704
Reading /raven/ptmp/fklotzsche/Experiments/vMemEcc/Data2022/DataMNE/EEG/03_preproc/ica/VME_S26-ica.fif ...
Now restoring ICA solution ...
Ready.
Not setting metadata
Not setting metadata
906 matching events found
No baseline correction applied
0 projection items activated
Loading data for 906 events and 500 original time points ...
11 bad epochs dropped
Created 93 VEOG epochs.
Created 59 HEOG epochs.
Using EOG channel: VEOG
Using EOG channel: HEOG
Applying ICA to Epochs instance
    Transforming to ICA space (62 components)
    Zeroing out 2 ICA components
    Projecting back using 62 PCA components
Applying ICA to Epochs instance
    Transforming to ICA space (62 components)
    Zeroing out 2 ICA components
    Projecting back using 62 PCA components
Applying ICA to Epochs instance
    Transforming to ICA space (62 components)
    Zeroing out 2 ICA components
    Projecting back using 62 PCA components
Opening raw data file /rav

  0%|          | Repairing epochs : 0/720 [00:00<?,       ?it/s]

Dropped 37 epochs: 5, 7, 8, 14, 45, 51, 54, 59, 61, 62, 63, 66, 86, 141, 161, 208, 248, 262, 285, 328, 333, 353, 376, 406, 424, 431, 451, 456, 492, 498, 546, 573, 600, 642, 646, 696, 699
Reading /raven/ptmp/fklotzsche/Experiments/vMemEcc/Data2022/DataMNE/EEG/03_preproc/ica/VME_S27-ica.fif ...
Now restoring ICA solution ...
Ready.
Not setting metadata
Not setting metadata
710 matching events found
No baseline correction applied
0 projection items activated
Loading data for 710 events and 500 original time points ...
6 bad epochs dropped
Created 139 VEOG epochs.
Created 20 HEOG epochs.
Not really a lot. This might be a problem! Consider marking longer stretches of clean EOG data.
########################
Using EOG channel: VEOG
Using EOG channel: HEOG
Applying ICA to Epochs instance
    Transforming to ICA space (62 components)
    Zeroing out 2 ICA components
    Projecting back using 62 PCA components
Applying ICA to Epochs instance
    Transforming to ICA space (62 components)
    Zer

In [21]:
post = ica.apply(data_forAR.copy()).crop(-config.times_dict['bl_dur_erp'],0)

In [22]:
pre = ica.apply(data_forICA).apply_baseline((-config.times_dict['bl_dur_erp'],0)).crop(-config.times_dict['bl_dur_erp'],0)

In [23]:
dpre=pre.get_data()
dpost=post.get_data()

In [19]:
dpre[0,0,:10]


array([8.30640721e-06, 8.83950717e-06, 9.62470637e-06, 1.04978965e-05,
       1.12398910e-05, 1.16156048e-05, 1.14184277e-05, 1.05187763e-05,
       8.90102502e-06, 6.68616894e-06])

In [20]:
dpost[0,0,:10]

array([2.58067086e-06, 3.11377082e-06, 3.89897001e-06, 4.77216016e-06,
       5.51415463e-06, 5.88986845e-06, 5.69269132e-06, 4.79303997e-06,
       3.17528866e-06, 9.60432589e-07])

In [32]:
np.argmin(np.mean(np.mean(dpre,axis=2),axis=0))

64

In [33]:
pre.ch_names[64]

'ECG'